**1.Перенос файлов на сервер**

In [ ]:
!scp /Users/mixanikkk/Downloads/Посещения.csv ubuntu@158.160.178.219:/tmp
!scp /Users/mixanikkk/Downloads/Bible.txt ubuntu@158.160.178.219:/tmp
!hadoop fs -put /tmp/Bible.txt /user/
!hadoop fs -put /tmp/Посещения.csv /user/




In [82]:
!hadoop fs -ls /user/

Found 9 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-11 16:05 /user/bible.txt
drwxr-xr-x   - hive   hadoop          0 2025-04-10 14:45 /user/hive
-rw-r--r--   1 ubuntu hadoop        258 2025-04-15 13:12 /user/mp.py
-rw-r--r--   1 ubuntu hadoop        491 2025-04-15 13:12 /user/reduce.py
drwxr-xr-x   - ubuntu hadoop          0 2025-04-15 13:37 /user/result_bible
drwxr-xr-x   - ubuntu hadoop          0 2025-04-10 16:03 /user/root
drwxr-xr-x   - ubuntu hadoop          0 2025-04-10 16:02 /user/ubuntu
drwxr-xr-x   - ubuntu hadoop          0 2025-04-15 14:32 /user/word_count_task
-rw-r--r--   1 ubuntu hadoop   36443383 2025-04-11 16:31 /user/Посещения.csv


In [84]:
!hadoop fs -cat /user/bible.txt | head -10

In the beginning God created the heaven and the earth. And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters. 
And God said, Let there be light: and there was light. 
And God saw the light, that it was good: and God divided the light from the darkness. 
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day. 
And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters. 
And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so. 
And God called the firmament Heaven. And the evening and the morning were the second day. 
And God said, Let the waters under the heaven be gathered together unto one place, and let the dry land appear: and it was so. 
And God called the dry land Earth; and the gathering together 

**2.1 Подсчет слов**

In [ ]:
%%writefile mp.py

import sys
import re

pattern = re.compile(r"[\S]{5,}")
def mapper():
    for line in sys.stdin:
        line = line.strip().lower()
        for match in pattern.finditer(line):
            print(f"{match.group()}\t1")
if __name__ == "__main__":
    mapper() 

In [19]:
%%writefile reduce.py
import sys

def reducer():
    word=next(sys.stdin).split('\t')[0]
    number =int(next(sys.stdin).split('\t')[1])
    for line in sys.stdin:
        cur_word, cur_number = line.split('\t')
        cur_number = int(cur_number)
        if cur_word!=word:
            print(f'{word}\t{number}')
            word = cur_word
            number = cur_number
        else:
            number+=cur_number               
    print(f'{word}\t{number}')     
if __name__ == "__main__":
    reducer() 

Overwriting reduce.py


In [71]:
%%time
! hadoop fs -cat /user/bible.txt | python3 mp.py | sort -k1| python3 reduce.py > final.txt

CPU times: user 71.3 ms, sys: 1.62 ms, total: 72.9 ms
Wall time: 3.48 s


In [77]:
cat final.txt| sort -k2,2 -n -r

shall	9733
which	4244
their	3859
there	2008
before	1722
lord,	1613
against	1596
shalt	1589
children	1560
said,	1556
them,	1549
saying,	1272
house	1222
every	1208
people	1194
because	1178
thee,	1170
these	1147
saith	1135
after	1128
behold,	1073
therefore	1054
israel	1025
among	907
thine	883
neither	861
great	847
brought	815
things	780
jesus	777
should	772
according	755
israel,	724
forth	720
bring	711
david	687
them.	674
lord.	674
heard	559
called	557
father	538
people,	512
spake	510
hundred	501
thee.	486
moses	481
given	476
heart	461
those	455
about	434
through	432
seven	432
would	430
three	426
might	425
other	415
speak	413
thousand	404
place	396
where	381
without	378
answered	377
under	376
pass,	374
house,	372
again	371
commanded	366
offering	361
first	358
thing	357
until	356
words	353
midst	347
toward	344
burnt	341
spirit	339
found	339
himself	328
earth	326
voice	323
judah	323
thereof	319
israel.	316
dwell	314
wherefore	311
cities	309
thereof,	308
whose	307
round	306
right	302
years	3

antioch.	5
antioch	5
answerest	5
anger;	5
angels.	5
andrew,	5
anak,	5
amoz,	5
amnon,	5
ammonite	5
amminadab	5
ammihud.	5
amend	5
ambush	5
amaziah,	5
amazed	5
amasai,	5
always.	5
alive?	5
alike	5
alienated	5
aholibamah	5
aholiab,	5
ahohite,	5
ahira	5
ahiezer	5
agagite,	5
after.	5
aforetime	5
afflictions	5
affairs	5
advice	5
adversity,	5
adversaries.	5
adultery:	5
adulterers,	5
adjure	5
acquaintance	5
achaia,	5
accusers	5
abundantly.	5
absalom's	5
abroad;	5
above;	5
abomination.	5
abishag	5
abiram,	5
ability	5
abidan	5
abiathar,	5
abdon	5
aaron;	5
zurishaddai.	4
zorah,	4
zorah	4
zophar	4
zippor,	4
zilpah	4
ziklag	4
zibeon,	4
zeresh	4
zelophehad,	4
zebulun.	4
zebadiah,	4
zebadiah	4
zarhites:	4
zacharias,	4
zabdi,	4
zabad,	4
zabad	4
yieldeth	4
yesterday,	4
yellow	4
years?	4
wrung	4
wrong:	4
wrest	4
woven	4
would.	4
would,	4
worthy,	4
worshipping	4
worshippeth	4
worse.	4
wormwood,	4
workman,	4
wool,	4
wondered,	4
wonder,	4
wizards,	4
witnessed	4
within.	4
withholdeth	4
withereth,	4
withered

blossom,	1
bloomed	1
bloodthirsty	1
bloodguiltiness,	1
blood?	1
blocks	1
block	1
blindness.	1
blindfolded	1
blindeth	1
blinded:	1
blinded.	1
blew.	1
blessest,	1
blessed?	1
bless;	1
bless:	1
blemishes,	1
blemishes	1
blemish)	1
bleatings	1
bleating	1
blaze	1
blastus	1
blasphemously	1
blaspheming.	1
blasphemies;	1
blasphemies:	1
blasphemeth;	1
blasphemeth.	1
blasphemest;	1
blasphemers,	1
blasphemers	1
blasphemer,	1
blasphemed?	1
blasphemed,	1
blaspheme;	1
blaspheme:	1
blaspheme.	1
blaspheme,	1
blameless?	1
blamed:	1
blamed.	1
blade;	1
blade	1
blackish	1
blacker	1
black.	1
biztha,	1
bizjothjah,	1
bitterness;	1
bittern,	1
bitterly:	1
bitter?	1
bitter!	1
bitten,	1
bitten	1
bithynia:	1
bithynia,	1
bithron,	1
bithiah	1
bishops	1
bishoprick	1
bishop,	1
bishlam,	1
birzavith.	1
birthright;	1
birthright,	1
birthday,	1
birsha	1
birds;	1
birds.	1
birds'	1
bird's	1
bird?	1
binea;	1
binea:	1
bimhal,	1
bilhan;	1
bilhan:	1
bilhah.	1
bilgai,	1
bileam	1
bigvai;	1
bigthana	1
bigthan	1
bigtha,	1
bier:	1
bie

In [105]:
!sudo find /usr/ -name hadoop-streaming.jar

/usr/lib/hadoop-mapreduce/hadoop-streaming.jar


In [79]:
!sudo mkdir -p /usr/lib/hadoop/logs
!ls -ld /usr/lib/hadoop/logs
!sudo chmod 0777  /usr/lib/hadoop/logs

drwxrwxrwx 2 root root 4096 Apr 15 12:54 /usr/lib/hadoop/logs


In [84]:
!sudo -u hdfs hdfs balancer -treshhold 5 

Usage: hdfs balancer
	[-policy <policy>]	the balancing policy: datanode or blockpool
	[-threshold <threshold>]	Percentage of disk capacity
	[-exclude [-f <hosts-file> | <comma-separated list of hosts>]]	Excludes the specified datanodes.
	[-include [-f <hosts-file> | <comma-separated list of hosts>]]	Includes only the specified datanodes.
	[-source [-f <hosts-file> | <comma-separated list of hosts>]]	Pick only the specified datanodes as source nodes.
	[-blockpools <comma-separated list of blockpool ids>]	The balancer will only run on blockpools included in this list.
	[-idleiterations <idleiterations>]	Number of consecutive idle iterations (-1 for Infinite) before exit.
	[-runDuringUpgrade]	Whether to run the balancer during an ongoing HDFS upgrade.This is usually not desired since it will not affect used space on over-utilized machines.

Apr 15, 2025 12:56:56 PM Balancing took 216.0 milliseconds
2025-04-15 12:56:56,234 ERROR balancer.Balancer: Exiting balancer due an exception
java.lan

In [101]:
!hdfs dfs -put mp.py /user/
!hdfs dfs -put reduce.py /user/



put: `/user/mp.py': File exists
put: `/user/reduce.py': File exists


In [118]:
!hdfs dfs -ls /user/mp.py



-rw-r--r--   1 ubuntu hadoop        258 2025-04-15 13:12 /user/mp.py


In [73]:
!hdfs dfs -rm -r /user/word_count_task
!yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name='wordcounter' \
-D mapreduce.job.reduces=3 \
-files mp.py,reduce.py \
-mapper "python3 mp.py map"\
-reducer "python3 reduce.py reducer"\
-input /user/bible.txt \
-output /user/word_count_task


Deleted /user/word_count_task
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob7744826009175359823.jar tmpDir=null
2025-04-15 14:31:58,194 INFO client.RMProxy: Connecting to ResourceManager at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:8032
2025-04-15 14:31:58,417 INFO client.AHSProxy: Connecting to Application History server at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:10200
2025-04-15 14:31:58,461 INFO client.RMProxy: Connecting to ResourceManager at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:8032
2025-04-15 14:31:58,461 INFO client.AHSProxy: Connecting to Application History server at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:10200
2025-04-15 14:31:58,675 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1744717336963_0015
2025-04-15 14:31:59,027 INFO mapred.FileInputFormat: Total input files to 

In [26]:
!hdfs dfs -ls /user/word_count_task/

Found 4 items
-rw-r--r--   1 ubuntu hadoop          0 2025-04-15 14:02 /user/word_count_task/_SUCCESS
-rw-r--r--   1 ubuntu hadoop      39821 2025-04-15 14:02 /user/word_count_task/part-00000
-rw-r--r--   1 ubuntu hadoop      40414 2025-04-15 14:02 /user/word_count_task/part-00001
-rw-r--r--   1 ubuntu hadoop      39435 2025-04-15 14:02 /user/word_count_task/part-00002


In [81]:
!hdfs dfs -cat /user/word_count_task/* | grep  -E 'lord\.|god\.|pray\.' | sort -t$'\t' -k2.2nr  | head -n 3

lord.	674
pray.	5


**Топ 5 сайтов по посещаемости**

In [87]:
!hadoop fs -cat /user/Посещения.csv | head -10











cat: Unable to write to output stream.


In [62]:
%%writefile mp_for_website.py

import sys
import re
import csv

def mapper():
    for row in csv.reader(sys.stdin):
        row= row[0]
        url = re.split(r"[; ]", row, maxsplit=0)
        print(f'{url[1]}+{url[0]}\t1')
if __name__ == "__main__":
    mapper() 

Overwriting mp_for_website.py


In [63]:
! hadoop fs -cat /user/Посещения.csv | python3 mp_for_website.py | head -10

2024-05-28+https://gonzales-bautista.com/	1
2024-05-26+https://gonzales-bautista.com/	1
2024-05-31+https://www.davis-berg.com/	1
2024-05-29+https://www.montoya.com/	1
2024-06-01+https://gonzales-bautista.com/	1
2024-05-28+https://www.davis-berg.com/	1
2024-05-27+https://gonzales-bautista.com/	1
2024-05-31+https://gonzales-bautista.com/	1
2024-05-27+https://www.davis-berg.com/	1
2024-05-27+https://gonzales-bautista.com/	1
Traceback (most recent call last):
  File "mp_for_website.py", line 12, in <module>
    mapper() 
  File "mp_for_website.py", line 10, in mapper
    print(f'{url[1]}+{url[0]}\t1')
BrokenPipeError: [Errno 32] Broken pipe
cat: Unable to write to output stream.


In [11]:
%%writefile reduce_for_website.py
import sys


def reducer():
    data_by_date = []
    website=next(sys.stdin).split('\t')[0]
    date =website.split('+')[0]
    number =int(next(sys.stdin).split('\t')[1])
    for line in sys.stdin:
        cur_website, cur_number = line.split('\t', 1)
        cur_date=cur_website.split("+")[0]
        cur_number = int(cur_number)
        if cur_website!=website:
            if cur_date==date:
                data_by_date.append((website, number))
            else:
                data_sorted=sorted(data_by_date, key=lambda x: x[1], reverse=True)
                for site, visits in data_sorted[:5]:
                    print(site.replace("+", ' ', 1), visits)
                data_by_date = []
                date = cur_date
            website = cur_website
            number = cur_number
        else:
            number+=cur_number
    data_by_date.append((website, number))
    data_sorted=sorted(data_by_date, key=lambda x: x[1], reverse=True)
    for site, visits in data_sorted[:5]:
        print(site.replace("+", ' ', 1), visits) 
if __name__ == "__main__":
    reducer() 

Overwriting reduce_for_website.py


Используем проверку на дату, если дата одиноковая но сайты различаются вносим сайт в список, как только дата меняется выбираем топ 5 сайтов за день

In [20]:
!hdfs dfs -rm -r /user/websites
!yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D stream.map.output.field.separator=+\
-D stream.num.map.output.key.fields=2 \
-D map.output.key.field.separator=+ \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapreduce.job.name='websites' \
-D mapreduce.job.reduces=3 \
-files mp_for_website.py,reduce_for_website.py \
-mapper "python3 mp_for_website.py map"\
-reducer "python3 reduce_for_website.py reducer"\
-input /user/Посещения.csv \
-output /user/websites\
-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner


Deleted /user/websites
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob1203960904276496227.jar tmpDir=null
2025-04-16 22:05:00,150 INFO client.RMProxy: Connecting to ResourceManager at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:8032
2025-04-16 22:05:00,386 INFO client.AHSProxy: Connecting to Application History server at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:10200
2025-04-16 22:05:00,435 INFO client.RMProxy: Connecting to ResourceManager at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:8032
2025-04-16 22:05:00,436 INFO client.AHSProxy: Connecting to Application History server at rc1d-dataproc-m-j7ha63uqu54o1bvr.mdb.yandexcloud.net/10.130.0.9:10200
2025-04-16 22:05:00,681 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1744823371038_0043
2025-04-16 22:05:01,074 INFO mapred.FileInputFormat: Total input files to process

Устанавливаем key.partitioner=k1,1, чтобы ключи разделялись по датам 

In [21]:
!hdfs dfs -cat /user/websites/* | grep -E '2024-05-28|2024-06-02|2024-05-30' | column -t -s$'\t'

2024-05-30 https://gonzales-bautista.com/ 353
2024-05-30 https://smith.com/ 246
2024-05-30 https://www.smith.com/ 239
2024-05-30 http://smith.com/ 229
2024-05-30 http://www.smith.com/ 225
2024-06-02 http://smith.com/ 7
2024-06-02 https://gonzales-bautista.com/ 7
2024-06-02 https://www.williams.com/ 6
2024-06-02 http://lee.com/ 5
2024-06-02 http://miller.com/ 5
2024-05-28 https://gonzales-bautista.com/ 368
2024-05-28 https://smith.com/ 256
2024-05-28 https://www.smith.com/ 251
2024-05-28 http://smith.com/ 224
2024-05-28 http://www.smith.com/ 204
